<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/nl_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yh3rvy9c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-yh3rvy9c
  Resolved https://github.com/huggingface/transformers to commit b074461ef0f54ce37c5239d30ee960ece28d11ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [25]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [26]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [27]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "nl", split="test")
#column_name = "audio, normalized_text"

In [28]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("medium")

In [29]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:28<00:00, 53.2MiB/s]


In [30]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [31]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="nl")
    return result["text"]

In [32]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [33]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score


In [34]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1

    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")
    # Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")


Processed 1 files, WER: 0.13333333333333333, Precision: 0.88, Recall: 0.88, F1: 0.88
Processed 2 files, WER: 0.3076923076923077, Precision: 0.75, Recall: 0.84, F1: 0.7924528301886793
Processed 3 files, WER: 0.1590909090909091, Precision: 0.8235294117647058, Recall: 0.8235294117647058, F1: 0.8235294117647058
Processed 4 files, WER: 0.2, Precision: 0.8, Recall: 0.8, F1: 0.8000000000000002
Processed 5 files, WER: 0.42105263157894735, Precision: 0.5625, Recall: 0.5294117647058824, F1: 0.5454545454545455
Processed 6 files, WER: 0.21428571428571427, Precision: 0.8571428571428571, Recall: 0.9230769230769231, F1: 0.888888888888889
Processed 7 files, WER: 0.15789473684210525, Precision: 0.8235294117647058, Recall: 0.8235294117647058, F1: 0.8235294117647058
Processed 8 files, WER: 0.16666666666666666, Precision: 0.9333333333333333, Recall: 0.8235294117647058, F1: 0.8749999999999999
Processed 9 files, WER: 0.4, Precision: 0.6470588235294118, Recall: 0.6111111111111112, F1: 0.6285714285714287
Proc